# Método ADI

Queremos resolver el siguiente problema utilizando un método explícito y luego el método ADI.

\begin{equation*}\left\{\begin{array}{cc}U_t = \sigma( U_{xx} +  U_{yy})  + a_x U_x + a_y U_y + f & \Omega = [0,1]^2 , \; t\in(0,T_f) \\
	U(x,y,t) = 0 & (x,y)\in\partial\Omega, \; t\in(0,T_f). \\
        U(x,y,0) = U_0(x,y) &  (x,y)\in\Omega\end{array}\right.\end{equation*}

## Datos del problema

* $f=1$
* $\sigma=1$
* $a_x=5$, $a_y=5$
* $U_0(x,y)=e^{-10((x-0.5)^2+(y-0.5)^2)}$
* $T_f = 0.5$

In [3]:
using LinearAlgebra, Plots

In [5]:
f(x,y) = 1
U₀(x,y) = exp(-10*((x-0.5)^2+(y-0.5)^2))
a_x = 5
a_y = 5
T_f = 0.5

0.5

In [7]:
function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end

meshgrid (generic function with 1 method)

In [9]:
function explicito(h,r;f=f,U₀=U₀,σ=1,a_x=5,a_y=5,T_f=0.5)
     
    if h*abs(a_x)>2*σ
      print("Cuidado! Es inestable")
    elseif h*abs(a_x)>2*σ
      print("Cuidado! Es inestable")
    elseif r>1/(4*σ)
      print("Cuidado! Es inestable")
    end

    dt=(h^2)*r
    N=round(T_f/dt)
    N=Int(N)

    #Genero la grilla de puntos
    x = h:h:1-h
    y = x

    X,Y = meshgrid(x,y)
    l = size(X,1)
    
    #Armamos las matrices para el metodo explicito
    A = (1/(h^2))*Tridiagonal(ones(l-1),-2*ones(l),ones(l-1))
    B = (1/(2*h))*Tridiagonal(-ones(l-1),zeros(l),ones(l-1))

    u = U₀.(X,Y)
    f = f.(X,Y)

    for j=1:N
        u = u+dt*((σ*A+a_x*B)*u+u*(σ*A+a_y*B')+f)
    end
    
    return X,Y,u
end


explicito (generic function with 1 method)

In [ ]:
surface(explicito(0.02,0.01))

In [13]:
function adi(h,r;f=f,U₀=U₀,σ=1,a_x=5,a_y=5,T_f=0.5)
    dt=(h^2)*r
    N=round(T_f/dt)
    N=Int(N)

    #Genero la grilla de puntos
    x = h:h:1-h
    y = x

    X,Y = meshgrid(x,y)
    l = size(X,1)
    
    #Armamos las matrices para el metodo explicito
    A = (1/(h^2))*Tridiagonal(ones(l-1),-2*ones(l),ones(l-1))
    B = (1/(2*h))*Tridiagonal(-ones(l-1),zeros(l),ones(l-1))

    u = U₀.(X,Y)
    f = f.(X,Y)

    ####Metodo ADI####
    Id = I(l)
    P1 = inv(Id-(dt/2)*(σ*A+a_x*B))
    P2 = inv(Id-(dt/2)*(σ*A+a_y*B'))

    for j=1:N
        u_inter = P1*(u'*(Id+(dt/2)*(σ*A+a_y*B'))+(dt/2)*f)
        u_adi = ((Id+(dt/2)*(σ*A+a_x*B))*u_inter+(dt/2)*f)*P2
        u = u_adi
    end
    
    return X,Y,u
end

adi (generic function with 1 method)

In [ ]:
surface(adi(0.02,0.01))

### Animación

In [17]:
function adi_animate(h,dt,pasos;f=f,U₀=U₀,σ=1,a_x=5,a_y=5,T_f=0.5)
    N=pasos
    
    #Genero la grilla de puntos
    x = h:h:1-h
    y = x

    X,Y = meshgrid(x,y)
    l = size(X,1)
    
    #Armamos las matrices para el metodo explicito
    A = (1/(h^2))*Tridiagonal(ones(l-1),-2*ones(l),ones(l-1))
    B = (1/(2*h))*Tridiagonal(-ones(l-1),zeros(l),ones(l-1))

    u = U₀.(X,Y)
    f = f.(X,Y)

    ####Metodo ADI####
    Id = I(l)
    P1 = inv(Id-(dt/2)*(σ*A+a_x*B))
    P2 = inv(Id-(dt/2)*(σ*A+a_y*B'))

    for j=1:N
        u_inter = P1*(u'*(Id+(dt/2)*(σ*A+a_y*B'))+(dt/2)*f)
        u_adi = ((Id+(dt/2)*(σ*A+a_x*B))*u_inter+(dt/2)*f)*P2
        u = u_adi
    end
    
    return X,Y,u
end

adi_animate (generic function with 1 method)

In [ ]:
pasos = 30
anim = @animate for i=1:pasos
    surface(adi_animate(0.02,0.01,i,σ=1),zlim=(0,1))
end
gif(anim, "anim.gif", fps = 2) 

### Veamos que pasa con la estabilidad

In [ ]:
surface(explicito(0.05,0.255))

In [ ]:
surface(adi(0.05,0.255))

### Ejercicio

El objetivo ahora es estimar el orden del método ADI para $\Delta t$. Para esto adaptar la función `adi` para que tome como parametro $\Delta t$ y luego crear una solución de referencia $u_{ref}$. Tomar distintos valores del paso y comparar el error con dicha función de referencia.

1. Hacer un gráfico $\log(\Delta t)$ vs $\log(error)$.
2. Usar el la función `fit` del paquete `Polynomials` para hacer la estimación del orden. 

In [49]:
## completar